In [ ]:
import os
import pandas as pd
path='...name/.csv'
data=pd.read_csv(path, error_bad_lines=False)
data.drop(data.index[6500:],inplace=True)

In [ ]:
images=[]
for i in data.id:
    img='images/'+str(i)+'.jpg'
    images.append(img)


In [ ]:
from sklearn.preprocessing import LabelEncoder
labencod=LabelEncoder()
labencod.fit(data.articleType)
data.articleType=labencod.transform(data.articleType)

In [ ]:
from keras.utils import to_categorical
labels=to_categorical(data.articleType)

In [ ]:
from sklearn.model_selection import train_test_split
tr_x,te_x,tr_y,te_y=train_test_split(images,labels,train_size=0.90)
image_train=[]
image_valid=[]

In [ ]:
for img in tr_x:
    image=decoder_images(img,60,60)
    images_train.append(image)

for img in te_x:
    image=decoder_images(img,60,60)
    images_valid.append(image)

In [ ]:
import .layers import convolution2d,max_pooling2d,fc_dense,out_dense,batch_normalization
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
x=tf.placeholder(tf.float32,name='x',shape=[None,60,60,3])
y=tf.placeholder(tf.int32,name='y',shape=[None,labels.shape[1]])

conv1_1 = convolution2d('conv1_1', x, 64, stride=[1, 1, 1, 1])
conv1_2 = convolution2d('conv1_2', conv1_1, 64, stride=[1, 1, 1, 1])
pool1 = max_pooling2d(pool1', conv1_2, kernel=[1, 2, 2, 1], stride=[1, 2, 2, 1])

conv2_1 = convolution2d('conv2_1', pool1, 128, stride=[1, 1, 1, 1])
conv2_2 = convolution2d('conv2_2', conv2_1, 128, stride=[1, 1, 1, 1])
pool2 = max_pooling2d('pool2', conv2_2, kernel=[1, 2, 2, 1], stride=[1, 2, 2, 1])

conv3_1 = convolution2d('conv3_1', pool2, 256, stride=[1, 1, 1, 1])
conv3_2 = convolution2d('conv3_2', conv3_1, 256, stride=[1, 1, 1, 1])
conv3_3 = convolution2d('conv3_3', conv3_2, 256, stride=[1, 1, 1, 1])
pool3 =max_pooling2d('pool3', conv3_3, kernel=[1, 2, 2, 1], stride=[1, 2, 2, 1])

conv4_1 = convolution2d('conv4_1', pool3, 512, stride=[1, 1, 1, 1])
conv4_2 = convolution2d('conv4_2', conv4_1, 512, stride=[1, 1, 1, 1])
conv4_3 = convolution2d('conv4_3', conv4_2, 512, stride=[1, 1, 1, 1])
pool4 = max_pooling2d('pool4', conv4_3, kernel=[1, 2, 2, 1], stride=[1, 2, 2, 1])

conv5_1 = convolution2d('conv5_1', pool4, 512, stride=[1, 1, 1, 1])
conv5_2 = convolution2d('conv5_2', conv5_1, 512, stride=[1, 1, 1, 1])
conv5_3 = convolution2d('conv5_3', conv5_2, 512, stride=[1, 1, 1, 1])
pool5 =max_pooling2d('pool5', conv5_3, kernel=[1, 2, 2, 1], stride=[1, 2, 2, 1])

fc6 = fc_dense('fc6', pool5, out_nodes=4096)
batch_norm1 = batch_normalization('batch_norm1', fc6)
fc7 = fc_dense('fc7', batch_norm1, out_nodes=4096)
batch_norm2 = batch_normalization('batch_norm2', fc7)
logits = out_dense('fc8', batch_norm2, out_nodes=labels.shape[1])

In [ ]:
from datetime import datetime
now = datetime.utcnow( ).strftime( "%Y%m%d%H%M%S" )
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,now)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=logits, labels=y, name='cross-entropy'))
optimizer =   tf.train.RMSPropOptimizer(learning_rate=0.00001,momentum=0.6,decay=0.7,epsilon=1e-10).minimize(cross_entropy )


mse_summary = tf.summary.scalar( 'cross-entropy' , cross_entropy  )
file_writer = tf.summary.FileWriter( logdir , tf.get_default_graph( ))

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sacer=tf.train.Saver()
numb=0
epochs=30
for i in range(epochs):
    numb+=10
    print('epoch',i)
    for j in range(len(tr_x)):
        x1=images_train[j]
        y1=tr_y[j]
        y1=np.expand_dims(y1,axis=0)
 
        summ=sess.run(mse_summary,feed_dict={x:x1,y:y1})
        file_writer.add_summary(summ, numb)
        sess.run(optimizer,feed_dict={x:x1,y:y1})
        
    loss=sess.run(cross_entropy,feed_dict={x:x1,y:y1})

    print('loss:=====================================',loss)
  
sacer_Path=sacer.save(sess,'path_name.ckpt')       

In [ ]:
with tf.Session() as sess:
    metrix_class=[]
    sacer.restore(sess,"path_name.ckpt")
   
   
    for j in range(len(te_x)):
        x1=images_valid[j]
    
        print('TEST TAG : ',te_y[j].argmax())
        a=tf.nn.softmax(logits).eval(feed_dict={x:x1})
        pred=np.argmax(a,axis=1)
        print('Predictions on test data : ',pred)
       